In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../")

import gurobipy
from json import dumps, loads
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score, recall_score)
from tqdm import tnrange, trange
import tensorflow as tf

from mlsql.influence import InfluenceRanker
from mlsql.fixer import AutoFixer
from mlsql.manager import ModelManagerLM
from mlsql.manager_test import ModelManagerTest

from models.simple_cnn import SimpleCNN
from models.logreg import LogReg
from models.linear_comb import LinearComb
from models.linear_comb_test import LinearCombTest
from processors.diabetes import DiabetesProcessor
from processors.diabetes_corruption import DiabetesCorrProcessor


import logging
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

import time
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
# @tf.function
def rank_fix(ranker, fixer, n):
    rank = ranker.predict()
    fixer.fix(rank, n)
    return rank

@tf.function
def rankit(ranker):
    rank = ranker.predict()
    return rank


@tf.function
def fixit(fixer, rank, n):
    fixer.fix(rank, n)


# @tf.function
def train(manager):
    manager.fit()

In [ ]:
proc = DiabetesCorrProcessor()

In [ ]:
# Rain
model = LogReg(1)
manager0 = ModelManagerLM(proc.x_train, proc.y_corr, model, 256)
start = time.time()
manager0.fit(print_value=True, tol=1e-7, max_iter=10000)
print(time.time() - start)
manager0.report(proc.x_train, proc.y_corr, proc.x_test, proc.y_test)
# manager0.report(proc.x_train, proc.y_corr, proc.x_query, proc.y_query)

In [ ]:
# print number of corruptions
corrsel = proc.corrsel
K = len(list(np.where(corrsel)[0]))
print(len(list(np.where(corrsel)[0])))

In [ ]:
# Rain Debug
from tqdm.notebook import tnrange, trange
manager = ModelManagerLM(proc.x_train, proc.y_corr, LogReg(1), 256)
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager, on=proc.complain)
# from mlsql.loss_ranker import LossRanker
# ranker = LossRanker(manager=manager_test)
fixer = AutoFixer(manager, corrsel, K)

AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
# AQ = proc.complain(manager).AQ
f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
# f1 = f1_score(proc.y_query.numpy(), manager.model.predict(proc.x_query).numpy(), average='weighted')
# AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 10
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    manager.fit(max_iter=5000, tol=1e-7, print_value=True)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

#     AQ = proc.complain(manager).AQ
    f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
#     f1 = f1_score(proc.y_query.numpy(), manager.model.predict(proc.x_query).numpy(), average='weighted')
#     AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
AC = proc.complain(manager).AC

df_rain_test = pd.DataFrame({
#     "Complain": np.array(AQs) - AC,
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("Rain", len(weighted_f1)),
})
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("Complain:Q"),
    color = "Method"
)

In [ ]:
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("F1:Q"),
    color = "Method"
)

In [ ]:
weighted_f1

In [ ]:
proc = DiabetesCorrProcessor()

In [ ]:
# Frog Model
model = LinearCombTest(1)
manager_test0 = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, model, 256)
start = time.time()
manager_test0.fit(print_value=True, tol=1e-8, lr=0.1, max_iter=20000)
print(time.time() - start)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_test, proc.x_b_test, proc.y_test)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_query, proc.x_b_query, proc.y_query)

In [ ]:
corrsel = proc.corrsel
K = len(list(np.where(corrsel)[0]))
print(len(list(np.where(corrsel)[0])))

In [ ]:
# Frog Debug
manager_test = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, LinearCombTest(1), 256)
manager_test.model.set_weights(manager_test0.model.get_weights())
manager_test.delta = tf.Variable(manager_test0.delta.value(), name="delta")
from mlsql.loss_ranker import LossRanker
# ranker = LossRanker(manager=manager_test)
ranker = InfluenceRanker(manager=manager_test, on=proc.test_complain)
fixer = AutoFixer(manager_test, proc.corrsel, K)

# AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
# AQ = proc.test_complain(manager_test).AQ
f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
# f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
# AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 10
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    manager_test.fit(max_iter=10000, tol=1e-8, print_value=True)

    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

#     AQ = proc.test_complain(manager_test).AQ
    f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
#     f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
#     AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
# AC = proc.test_complain(manager_test).AC


alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("F1:Q"),
    color = "Method"
)

In [ ]:
fixer.deletions

In [ ]:
np.where(proc.corrsel)

In [ ]:
fixer.recall_k()

In [ ]:
f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')

In [ ]:
proc = DiabetesCorrProcessor()
model = LinearCombTest(1)
manager_test0 = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, model, 256)
start = time.time()
manager_test0.fit(print_value=True, tol=1e-8, lr=0.1, max_iter=20000)
print(time.time() - start)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_test, proc.x_b_test, proc.y_test)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_query, proc.x_b_query, proc.y_query)

In [ ]:
# Loss Ranker
manager_test = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, LinearCombTest(1), 256)
manager_test.model.set_weights(manager_test0.model.get_weights())
manager_test.delta = tf.Variable(manager_test0.delta.value(), name="delta")
from mlsql.loss_ranker import LossRanker
ranker = LossRanker(manager=manager_test)
# ranker = InfluenceRanker(manager=manager_test, on=proc.test_complain)
fixer = AutoFixer(manager_test, proc.corrsel, K)

# AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
# AQ = proc.test_complain(manager_test).AQ
f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
# f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
# AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 10
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    manager_test.fit(max_iter=10000, tol=1e-8, print_value=True)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

#     AQ = proc.test_complain(manager_test).AQ
    f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
#     f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
#     AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
# AC = proc.test_complain(manager_test).AC

df_rain_test = pd.DataFrame({
#     "Complain": np.array(AQs) - AC,
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("Rain", len(weighted_f1)),
})
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("F1:Q"),
    color = "Method"
)